***GENERATED CODE FOR shippingpricemodel130921 PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'OrderDate', 'transformation_label': 'String Indexer'}], 'feature': 'OrderDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '11', 'mean': '', 'stddev': '', 'min': '2020-06-29 06:00', 'max': '2020-10-13 23:33', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'OrderDate'}, {'feature_label': 'OrderDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('OrderDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Carrier', 'transformation_label': 'String Indexer'}], 'feature': 'Carrier', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '11', 'mean': '', 'stddev': '', 'min': 'shopify', 'max': 'usps', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Carrier'}, {'feature_label': 'Carrier', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Carrier')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LineItemId', 'transformation_label': 'String Indexer'}], 'feature': 'LineItemId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '11', 'mean': '', 'stddev': '', 'min': '013B4121-C785-4FB1-A810-0F443B10FAFD', 'max': 'F0F94484-696D-4AE9-97A9-28665D6C858B', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LineItemId'}, {'feature_label': 'LineItemId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LineItemId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'OrderId', 'transformation_label': 'String Indexer'}], 'feature': 'OrderId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '11', 'mean': '', 'stddev': '', 'min': '4DA2C0D5-FD13-4BBB-9B0F-23DA2C403B39', 'max': 'F8CBB3C8-4BA0-4B86-BE4C-5068EED64B52', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'OrderId'}, {'feature_label': 'OrderId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('OrderId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProductName', 'transformation_label': 'String Indexer'}], 'feature': 'ProductName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '11', 'mean': '', 'stddev': '', 'min': 'Black Diamond 10 MM Dynex Runners - 120', 'max': "Scarpa HELIX WOMEN'S - 41", 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProductName'}, {'feature_label': 'ProductName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProductName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SKU', 'transformation_label': 'String Indexer'}], 'feature': 'SKU', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '11', 'mean': '3340615.33', 'stddev': '5767928.23', 'min': '10000845', 'max': 'R45AY 120', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SKU'}, {'feature_label': 'SKU', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SKU')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Ref_OrderLineItemId', 'transformation_label': 'String Indexer'}], 'feature': 'Ref_OrderLineItemId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '11', 'mean': '', 'stddev': '', 'min': 'LineItemRef_5552273195169', 'max': 'LineItemRef_5985294614689', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Ref_OrderLineItemId'}, {'feature_label': 'Ref_OrderLineItemId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Ref_OrderLineItemId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ToZip', 'transformation_label': 'String Indexer'}], 'feature': 'ToZip', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '11', 'mean': '80840.09', 'stddev': '26372.25', 'min': '12047', 'max': '98203', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ToZip'}, {'feature_label': 'ToZip', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ToZip')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProductId', 'transformation_label': 'String Indexer'}], 'feature': 'ProductId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '11', 'mean': '', 'stddev': '', 'min': '20B3D76826274DF587F6EBD66F5A7C3D', 'max': 'DC5D8592EEDE4428B25041E637D9A16F', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProductId'}, {'feature_label': 'ProductId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProductId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FulfillmentLineItemId', 'transformation_label': 'String Indexer'}], 'feature': 'FulfillmentLineItemId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '11', 'mean': '', 'stddev': '', 'min': '1469AB9D-3ED3-4DA8-96A0-5116101F59F7', 'max': 'F5D99553-B8D2-48DF-97F8-CCB572C6FB79', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FulfillmentLineItemId'}, {'feature_label': 'FulfillmentLineItemId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FulfillmentLineItemId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'OrderLineItemId', 'transformation_label': 'String Indexer'}], 'feature': 'OrderLineItemId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '11', 'mean': '', 'stddev': '', 'min': '013B4121-C785-4FB1-A810-0F443B10FAFD', 'max': 'F0F94484-696D-4AE9-97A9-28665D6C858B', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'OrderLineItemId'}, {'feature_label': 'OrderLineItemId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('OrderLineItemId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FulfillmentId', 'transformation_label': 'String Indexer'}], 'feature': 'FulfillmentId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '11', 'mean': '', 'stddev': '', 'min': '1BB172FE-4580-459E-8CD2-224CBA72D4D7', 'max': 'EEF01D4D-999B-496E-998E-A5E60EB9CECA', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FulfillmentId'}, {'feature_label': 'FulfillmentId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FulfillmentId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CarrierPickedUpDateTime', 'transformation_label': 'String Indexer'}], 'feature': 'CarrierPickedUpDateTime', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '11', 'mean': '', 'stddev': '', 'min': '2020-06-29 06:00', 'max': '2021-02-09 07:21', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CarrierPickedUpDateTime'}, {'feature_label': 'CarrierPickedUpDateTime', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CarrierPickedUpDateTime')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CarrierDeliveredDateTime', 'transformation_label': 'String Indexer'}], 'feature': 'CarrierDeliveredDateTime', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '11', 'mean': '', 'stddev': '', 'min': '2020-02-08 23:59', 'max': '2021-03-09 07:21', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CarrierDeliveredDateTime'}, {'feature_label': 'CarrierDeliveredDateTime', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'CarrierDeliveredDateTime')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


def randomforestregressor(df, labels, features):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    rf_model = RandomForestRegressor()
    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[labels])
    rf_model.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % rf_model.score(X_test, y_test))
    data = {'model': rf_model,
            'X_test': X_test,
            'y_test': y_test,
            'label': labels,
            'columnNames': df.columns}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run shippingpricemodel130921Hooks.ipynb
try:
	#sourcePreExecutionHook()

	shippingpricedata = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/1631522542684_ShippingCostData_130921.csv', 'filename': '1631522542684_ShippingCostData_130921.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/BiglynxUpdated/ShippingPriceData/ShippingCostData_130921.csv', 'viewFileName': 'ShippingCostData_130921.csv', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/', 'results_url': 'http://3.140.115.35:4040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run shippingpricemodel130921Hooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(shippingpricedata,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "OrderDate", "transformation_label": "String Indexer"}], "feature": "OrderDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "11", "mean": "", "stddev": "", "min": "2020-06-29 06:00", "max": "2020-10-13 23:33", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "OrderDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ShippingId", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "11", "mean": "2.09", "stddev": "0.54", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "ShippingId"}, {"transformationsData": [{"feature_label": "Carrier", "transformation_label": "String Indexer"}], "feature": "Carrier", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "11", "mean": "", "stddev": "", "min": "shopify", "max": "usps", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Carrier"}, {"transformationsData": [{"feature_label": "LineItemId", "transformation_label": "String Indexer"}], "feature": "LineItemId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "11", "mean": "", "stddev": "", "min": "013B4121-C785-4FB1-A810-0F443B10FAFD", "max": "F0F94484-696D-4AE9-97A9-28665D6C858B", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LineItemId"}, {"transformationsData": [{"feature_label": "OrderId", "transformation_label": "String Indexer"}], "feature": "OrderId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "11", "mean": "", "stddev": "", "min": "4DA2C0D5-FD13-4BBB-9B0F-23DA2C403B39", "max": "F8CBB3C8-4BA0-4B86-BE4C-5068EED64B52", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "OrderId"}, {"transformationsData": [{"feature_label": "ProductName", "transformation_label": "String Indexer"}], "feature": "ProductName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "11", "mean": "", "stddev": "", "min": "Black Diamond 10 MM Dynex Runners - 120", "max": "Scarpa HELIX WOMEN'S - 41", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProductName"}, {"transformationsData": [{"feature_label": "SKU", "transformation_label": "String Indexer"}], "feature": "SKU", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "11", "mean": "3340615.33", "stddev": "5767928.23", "min": "10000845", "max": "R45AY 120", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SKU"}, {"transformationsData": [{"feature_label": "Ref_OrderLineItemId", "transformation_label": "String Indexer"}], "feature": "Ref_OrderLineItemId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "11", "mean": "", "stddev": "", "min": "LineItemRef_5552273195169", "max": "LineItemRef_5985294614689", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Ref_OrderLineItemId"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Qty", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "11", "mean": "2.45", "stddev": "4.5", "min": "1", "max": "16", "missing": "0"}, "updatedLabel": "Qty"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Length", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "11", "mean": "5.09", "stddev": "2.59", "min": "1", "max": "9", "missing": "0"}, "updatedLabel": "Length"}, {"transformationsData": [{"feature_label": "ToZip", "transformation_label": "String Indexer"}], "feature": "ToZip", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "11", "mean": "80840.09", "stddev": "26372.25", "min": "12047", "max": "98203", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ToZip"}, {"transformationsData": [{"feature_label": "ProductId", "transformation_label": "String Indexer"}], "feature": "ProductId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "11", "mean": "", "stddev": "", "min": "20B3D76826274DF587F6EBD66F5A7C3D", "max": "DC5D8592EEDE4428B25041E637D9A16F", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProductId"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FromZip", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "11", "mean": "98052.0", "stddev": "0.0", "min": "98052", "max": "98052", "missing": "0"}, "updatedLabel": "FromZip"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RetailerId", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "11", "mean": "375544787.0", "stddev": "0.0", "min": "375544787", "max": "375544787", "missing": "0"}, "updatedLabel": "RetailerId"}, {"transformationsData": [{"feature_label": "FulfillmentLineItemId", "transformation_label": "String Indexer"}], "feature": "FulfillmentLineItemId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "11", "mean": "", "stddev": "", "min": "1469AB9D-3ED3-4DA8-96A0-5116101F59F7", "max": "F5D99553-B8D2-48DF-97F8-CCB572C6FB79", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FulfillmentLineItemId"}, {"transformationsData": [{"feature_label": "OrderLineItemId", "transformation_label": "String Indexer"}], "feature": "OrderLineItemId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "11", "mean": "", "stddev": "", "min": "013B4121-C785-4FB1-A810-0F443B10FAFD", "max": "F0F94484-696D-4AE9-97A9-28665D6C858B", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "OrderLineItemId"}, {"transformationsData": [{"feature_label": "FulfillmentId", "transformation_label": "String Indexer"}], "feature": "FulfillmentId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "11", "mean": "", "stddev": "", "min": "1BB172FE-4580-459E-8CD2-224CBA72D4D7", "max": "EEF01D4D-999B-496E-998E-A5E60EB9CECA", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FulfillmentId"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "quantity", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "11", "mean": "2.45", "stddev": "4.5", "min": "1", "max": "16", "missing": "0"}, "updatedLabel": "quantity"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Weight", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "11", "mean": "1.34", "stddev": "2.26", "min": "0.14", "max": "7.68", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Weight"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Volume", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "11", "mean": "1078.91", "stddev": "1592.82", "min": "30", "max": "5120", "missing": "0"}, "updatedLabel": "Volume"}, {"transformationsData": [{"feature_label": "CarrierPickedUpDateTime", "transformation_label": "String Indexer"}], "feature": "CarrierPickedUpDateTime", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "11", "mean": "", "stddev": "", "min": "2020-06-29 06:00", "max": "2021-02-09 07:21", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CarrierPickedUpDateTime"}, {"transformationsData": [{"feature_label": "CarrierDeliveredDateTime", "transformation_label": "String Indexer"}], "feature": "CarrierDeliveredDateTime", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "11", "mean": "", "stddev": "", "min": "2020-02-08 23:59", "max": "2021-03-09 07:21", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CarrierDeliveredDateTime"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ShipCost", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "11", "mean": "4.55", "stddev": "1.57", "min": "1.0", "max": "7.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ShipCost"}, {"feature": "OrderDate_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "11", "mean": "4.09", "stddev": "3.18", "min": "0.0", "max": "9.0", "missing": "0"}, "updatedLabel": "OrderDate_stringindexer_t..."}, {"feature": "Carrier_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "11", "mean": "0.55", "stddev": "0.82", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "Carrier_stringindexer_tra..."}, {"feature": "LineItemId_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "11", "mean": "4.09", "stddev": "3.18", "min": "0.0", "max": "9.0", "missing": "0"}, "updatedLabel": "LineItemId_stringindexer_..."}, {"feature": "OrderId_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "11", "mean": "4.09", "stddev": "3.18", "min": "0.0", "max": "9.0", "missing": "0"}, "updatedLabel": "OrderId_stringindexer_tra..."}, {"feature": "ProductName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "11", "mean": "3.36", "stddev": "2.84", "min": "0.0", "max": "8.0", "missing": "0"}, "updatedLabel": "ProductName_stringindexer..."}, {"feature": "SKU_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "11", "mean": "3.36", "stddev": "2.84", "min": "0.0", "max": "8.0", "missing": "0"}, "updatedLabel": "SKU_stringindexer_transfo..."}, {"feature": "Ref_OrderLineItemId_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "11", "mean": "4.09", "stddev": "3.18", "min": "0.0", "max": "9.0", "missing": "0"}, "updatedLabel": "Ref_OrderLineItemId_strin..."}, {"feature": "ToZip_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "11", "mean": "4.09", "stddev": "3.18", "min": "0.0", "max": "9.0", "missing": "0"}, "updatedLabel": "ToZip_stringindexer_trans..."}, {"feature": "ProductId_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "11", "mean": "3.36", "stddev": "2.84", "min": "0.0", "max": "8.0", "missing": "0"}, "updatedLabel": "ProductId_stringindexer_t..."}, {"feature": "FulfillmentLineItemId_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "11", "mean": "4.09", "stddev": "3.18", "min": "0.0", "max": "9.0", "missing": "0"}, "updatedLabel": "FulfillmentLineItemId_str..."}, {"feature": "OrderLineItemId_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "11", "mean": "4.09", "stddev": "3.18", "min": "0.0", "max": "9.0", "missing": "0"}, "updatedLabel": "OrderLineItemId_stringind..."}, {"feature": "FulfillmentId_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "11", "mean": "4.09", "stddev": "3.18", "min": "0.0", "max": "9.0", "missing": "0"}, "updatedLabel": "FulfillmentId_stringindex..."}, {"feature": "CarrierPickedUpDateTime_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "11", "mean": "2.55", "stddev": "2.62", "min": "0.0", "max": "7.0", "missing": "0"}, "updatedLabel": "CarrierPickedUpDateTime_s..."}, {"feature": "CarrierDeliveredDateTime_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "11", "mean": "2.55", "stddev": "2.62", "min": "0.0", "max": "7.0", "missing": "0"}, "updatedLabel": "CarrierDeliveredDateTime_..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run shippingpricemodel130921Hooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=randomforestregressor(autofe, ["OrderDate_stringindexer", "ShippingId", "Carrier_stringindexer", "LineItemId_stringindexer", "OrderId_stringindexer", "ProductName_stringindexer", "Ref_OrderLineItemId_stringindexer", "Qty", "Length", "ToZip_stringindexer", "ProductId_stringindexer", "FromZip", "RetailerId", "FulfillmentLineItemId_stringindexer", "OrderLineItemId_stringindexer", "FulfillmentId_stringindexer", "quantity", "Weight", "Volume", "CarrierPickedUpDateTime_stringindexer", "CarrierDeliveredDateTime_stringindexer"], "ShipCost")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
